In [48]:
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR

In [56]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


In [57]:
# define input size, hidden layer size, output size
D_i, D_k, D_o = 100, 4000, 50
# create model with two hidden layers
model = nn.Sequential(
       nn.Linear(D_i, D_k),
       nn.ReLU(),
       nn.Linear(D_k, D_k),
       nn.ReLU(),
       nn.Linear(D_k, D_o)).to(device)

In [58]:
# He initialization of weights
def weights_init(layer_in):
    if isinstance(layer_in, nn.Linear):
        nn.init.kaiming_uniform(layer_in.weight)
        #layer_in.weight.data.fill_(-10.0)
        #layer_in.bias.data.fill_(-100.0)
model.apply(weights_init)

/tmp/ipykernel_4434/3350313036.py:4: FutureWarning: `nn.init.kaiming_uniform` is now deprecated in favor of `nn.init.kaiming_uniform_`.
  nn.init.kaiming_uniform(layer_in.weight)


Sequential(
  (0): Linear(in_features=100, out_features=4000, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4000, out_features=4000, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4000, out_features=50, bias=True)
)

In [59]:
# choose least squares loss function
criterion = nn.MSELoss()
# construct SGD optimizer and initialize learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)
# object that decreases learning rate by half every 10 epochs
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [60]:
# create 100 random data points and store in data loader class
x = torch.randn(1000, D_i).to(device)
y = torch.randn(1000, D_o).to(device)
data_loader = DataLoader(TensorDataset(x,y), batch_size=10, shuffle=True)

In [61]:
# loop over the dataset 100 times
def train():
    for epoch in range(10):
        epoch_loss = 0.0
        # loop over batches
        for i, data in enumerate(data_loader):
            # retrieve inputs and labels for this batch
            x_batch, y_batch = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward pass
            pred = model(x_batch)
            loss = criterion(pred, y_batch)
            # backward pass
            loss.backward()
            # SGD update
            optimizer.step()
            # update statistics
            epoch_loss += loss.item()
        # print error
        print(f'Epoch {epoch:5d}, loss {epoch_loss:.3f}')
        # tell scheduler to consider updating learning rate
        scheduler.step()

In [62]:
%timeit train()

Epoch     0, loss 383.451
Epoch     1, loss 103.281
Epoch     2, loss 99.026
Epoch     3, loss 98.224
Epoch     4, loss 97.889
Epoch     5, loss 97.744
Epoch     6, loss 97.951
Epoch     7, loss 98.162
Epoch     8, loss 98.332
Epoch     9, loss 98.625
Epoch     0, loss 98.586
Epoch     1, loss 98.402
Epoch     2, loss 98.147
Epoch     3, loss 98.015
Epoch     4, loss 97.902
Epoch     5, loss 97.859
Epoch     6, loss 97.819
Epoch     7, loss 97.788
Epoch     8, loss 97.774
Epoch     9, loss 97.715
Epoch     0, loss 97.555
Epoch     1, loss 97.445
Epoch     2, loss 97.411
Epoch     3, loss 97.394
Epoch     4, loss 97.390
Epoch     5, loss 97.378
Epoch     6, loss 97.364
Epoch     7, loss 97.350
Epoch     8, loss 97.355
Epoch     9, loss 97.342
Epoch     0, loss 97.263
Epoch     1, loss 97.252
Epoch     2, loss 97.250
Epoch     3, loss 97.245
Epoch     4, loss 97.245
Epoch     5, loss 97.240
Epoch     6, loss 97.242
Epoch     7, loss 97.229
Epoch     8, loss 97.236
Epoch     9, loss 97.22